In [31]:
## Use DASH to build user GUI
# %pip install dash

from dash import Dash, html, dcc, callback, Output, Input, ctx
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt

from presets import setmedium

In [32]:
press_data = pd.read_csv(r"C:\Users\geral\OneDrive\Documents\GitHub\TempCompliance4HIFU\src\sample_data.csv")  # sample_data == 121 x 501 double
mediumProp = setmedium.setMedium('Water',0)
print(mediumProp)


{'name': 'Water', 'speed': 1500, 'density': 1000, 'absCoeff': 0.025, 'specHeatCap': 4180, 'thermDiff': 1.4599999999999998e-07}


In [33]:
# help(html.Div)

In [34]:
app = Dash('tempCompliance4HIFU')

# ALLOWED_TYPES = ("text", "number", "password", "email", "search","tel", "url", "range", "hidden",)

app.layout = [
    # Title
    html.H1(children = 'Temperature Compliance for HIFU Transducers', style = {'textAlign':'center'}), 
    html.H2(children = 'By Gerald Lee', style = {'textAlign':'center'}),

    # TOP Section - Button and Messages
    html.Div([
        html.Button('CALCULATE PRESSURE AND TEMPERATURE', id='button', n_clicks=0),
        html.Div(id='message_Display',style={'textAlign':'left'}),
    ],style={'width': '50%', 'float':'right', 'display': 'inline-block','padding': '10px 5px'}),
    
    
    # RIGHT Section - DISPLAYS
    html.Div([
        html.H2('Results', style ={'textAlign':'center'}), 
            dcc.Dropdown(['Pressure','Intensity','Temperature'],'Intensity',id='DROP_field2D'),
            dcc.Graph(id='GRAPH_field2D',hoverData={'points': [{'customdata': 'Pressure'}]}),
            dcc.Graph(id='GRAPH_time1D'),
    ], style={'width': '60%', 'float': 'right', 'display': 'inline-block'}),

    # SPACE
    html.Div([
        html.H3(' ',style ={'textAlign':'center'})
    ],style={'width':'3%', 'float': 'right', 'display': 'inline-block'}),

    # LEFT Section - INPUTS
    html.Div([     
            
        # Medium Section (Top)
        html.H3('Medium', style ={'textAlign':'center'}), 
        dcc.Dropdown(['Custom','Water','Glycerol','Egg White','Castor Oil'],'Water',id='DROP_medium'),
        
        html.P('Speed of Sound [m/s]', style ={'textAlign':'left'}), 
        dcc.Input(id="Speed", type='number', placeholder='Input Speed of Sound'),

        html.P('Density [kg/m^3]', style ={'textAlign':'left'}),
        dcc.Input(id="Density", type='number', placeholder='Input Density'),

        html.P('Absorption Coeffient [Np/(m*MHz^2)]', style ={'textAlign':'left'}),
        dcc.Input(id="AbsCoeff", type='number', placeholder='Input Absorption Coefficient'),
    
        html.P('Specific Heat Capacity [J/(kg*K)]', style ={'textAlign':'left'}),
        dcc.Input(id="SpecHeatCap", type='number', placeholder='Input Specific Heat Capacity'),
            
        html.P('Thermal Diffusivity [(m^2)/s]', style ={'textAlign':'left'}),
        dcc.Input(id="ThermDiff", type='number', placeholder='Input Thermal Diffusivity'),  
        

        # Trandsucer Section (Middle)
        html.H3('Transducer', style ={'textAlign':'center'}), 

        html.P('Frequency [MHz]', style ={'textAlign':'left'}),
        dcc.Input(id="Frequency", type='number', placeholder='Input Frequency'),

        html.P('F-Number', style ={'textAlign':'left'}),
        dcc.Input(id="FNum", type='number', placeholder='Input F Number'),

        html.P('Radius [mm]', style ={'textAlign':'left'}),
        dcc.Input(id="Radius", type='number', placeholder='Input Radius'),


        # Heating Section (Right)
        html.H3('Heating', style ={'textAlign':'center'}), 

        html.P('Time Heating [s]', style ={'textAlign':'left'}),
        dcc.Input(id="HeatTime", type='number', placeholder='Input Heat Time'),

        html.P('Time Cooling [s]', style ={'textAlign':'left'}),
        dcc.Input(id="CoolTime", type='number', placeholder='Input Cool Time'),

        html.P('Duty Cycle [Percentage]', style ={'textAlign':'left'}),
        dcc.Input(id="DutyCycle", type='number', placeholder='Input Duty Cycle'),


    ], style={'width': '15%', 'float': 'right', 'display': 'inline-block'}),

]


@callback(
    Output('Speed','value'),
    Output('Density','value'),
    Output('AbsCoeff','value'),
    Output('SpecHeatCap','value'),
    Output('ThermDiff','value'),
    Input('DROP_medium','value'),
    Input('Speed','value'),
    Input('Density','value'),
    Input('AbsCoeff','value'),
    Input('SpecHeatCap','value'),
    Input('ThermDiff','value'),
)
def getMedium(dropdown, speed, dens, abscoeff, heatcap, thermdiff):
    if speed == None: speed = 0
    if dens == None: dens = 0
    if abscoeff == None: abscoeff = 0
    if heatcap == None: heatcap = 0
    if thermdiff == None: thermdiff = 0
    userIn = dict(speed = speed,density = dens,absCoeff = abscoeff,specHeatCap = heatcap,thermDiff = thermdiff)
    mediumProp = setmedium.setMedium(dropdown, userIn)
    if mediumProp['error'] == 1:
        print('ERROR: getMedium and/or setmedium')
    else:
        return mediumProp['speed'], mediumProp['density'], mediumProp['absCoeff'], mediumProp['specHeatCap'], mediumProp['thermDiff']



@callback(
    Output('message_Display', 'children'),
    Input('button','n_clicks')
)
def update_message(button):
    if 'button' == ctx.triggered_id:
        msg = 'Processing...'
    else:
        msg = 'This is a Bad Text Message to Recieve'
    return html.Div(msg)



# @callback(
#     Output('GRAPH_field2D','figure'),
#     Input('DROP_field2D','value')
# )
# def update_figure(value, press_data, temp_data):
#     match value:
#         case 'Intensity':
#             display_array = press_data^2
#         case 'Pressure':
#             display_array = press_data
#         case 'Temperature':
#             display_array = temp_data

#     fig, ax = plt.imshow(
#             x = display_array,
#             hover_name='X = {}, R = {}'
#             )
#     ax.set_xlabel('Axial Distance (Z-Axis) [m]')
#     ax.set_ylabel('Radial Distance (R-Axis) [m]')
#     fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')
#     return fig

if __name__ == '__main__':
    app.run(debug=True)


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[34], line 105, in getMedium(
    dropdown='Water',
    speed=0,
    dens=0,
    abscoeff=0,
    heatcap=0,
    thermdiff=0
)
    103 userIn = dict(speed = speed,density = dens,absCoeff = abscoeff,specHeatCap = heatcap,thermDiff = thermdiff)
    104 mediumProp = setmedium.setMedium(dropdown, userIn)
--> 105 if mediumProp['error'] == 1:
        mediumProp = {'name': 'Water', 'speed': 1500, 'density': 1000, 'absCoeff': 0.025, 'specHeatCap': 4180, 'thermDiff': 1.4599999999999998e-07}
    106     print('ERROR: getMedium and/or setmedium')
    107 else:

KeyError: 'error'

